## Подсчёт пальцев

In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json

PERIOD = 120

# Read trackbar data
trackbar_dict = {}
with open("trackbar_memory.json", "r") as outfile:
     trackbar_dict = json.load(outfile)

def nothing(i):
    pass

# Creating trackbars
cv2.namedWindow("mask")
cv2.createTrackbar("low_hue", "mask", trackbar_dict['low_hue'], 255, nothing)
cv2.createTrackbar("low_saturation", "mask", trackbar_dict['low_saturation'], 255, nothing)
cv2.createTrackbar("low_value", "mask", trackbar_dict['low_value'], 255, nothing)
cv2.createTrackbar("high_hue", "mask", trackbar_dict['high_hue'], 255, nothing)
cv2.createTrackbar("high_saturation", "mask", trackbar_dict['high_saturation'], 255, nothing)
cv2.createTrackbar("high_value", "mask", trackbar_dict['high_value'], 255, nothing)
cv2.createTrackbar("kernel", "mask", trackbar_dict['kernel'], 255, nothing)

video_path = "fingers.mov"
cam = cv2.VideoCapture(video_path)

pause_flag = False

while True:
    if not pause_flag: 
        success, frame = cam.read()
        outframe = frame.copy()

    if not success:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Calibrate mask
    trackbar_dict['low_hue'] = cv2.getTrackbarPos("low_hue", "mask")
    trackbar_dict['low_saturation'] = cv2.getTrackbarPos("low_saturation", "mask")
    trackbar_dict['low_value'] = cv2.getTrackbarPos("low_value", "mask")
    trackbar_dict['high_hue'] = cv2.getTrackbarPos("high_hue", "mask")
    trackbar_dict['high_saturation'] = cv2.getTrackbarPos("high_saturation", "mask")
    trackbar_dict['high_value'] = cv2.getTrackbarPos("high_value", "mask")
    trackbar_dict['kernel'] = cv2.getTrackbarPos("kernel", "mask")
    mask = cv2.inRange(hsv, (trackbar_dict['low_hue'], trackbar_dict['low_saturation'], trackbar_dict['low_value']), 
                       (trackbar_dict['high_hue'], trackbar_dict['high_saturation'], trackbar_dict['high_value']))
    
    # Cleaning erosion
    kernel = np.ones((3, 3), dtype=np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_ERODE, kernel, iterations=3)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=3)
    
    # Morphological detecting
    kersize = trackbar_dict['kernel'] * 2 + 1
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kersize, kersize))
    morph = cv2.morphologyEx(mask, cv2.MORPH_ERODE, kernel, iterations=3)
    morph = cv2.morphologyEx(morph, cv2.MORPH_DILATE, kernel, iterations=6)
    fingers = cv2.subtract(mask, morph)
    kernel = np.ones((5, 5), dtype=np.uint8)
    fingers = cv2.morphologyEx(fingers, cv2.MORPH_ERODE, kernel, iterations=4)
    fingers = cv2.morphologyEx(fingers, cv2.MORPH_DILATE, kernel, iterations=1)

    # Analysing connected components
    output = cv2.connectedComponentsWithStats(fingers, connectivity = 4)
    num_components = output[0]
    stats = output[2]
    num = 0

    for i in range(num_components):
        top    = stats[i, cv2.CC_STAT_TOP]
        left   = stats[i, cv2.CC_STAT_LEFT]
        width  = stats[i, cv2.CC_STAT_WIDTH]
        height = stats[i, cv2.CC_STAT_HEIGHT]
        area   = stats[i, cv2.CC_STAT_AREA]

        if area > 400 and area < 30000:
            num += 1
            cv2.rectangle(outframe, (left, top), (left + width, top + height), (20, 255, 20), 3)
            # cv2.rectangle(fingers, (left, top), (left + width, top + height), (155), 3)
        
    cv2.putText(outframe, "fingers number: " + str(num), (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (20, 255, 20), 2)
    cv2.imshow("image", outframe)
    cv2.imshow("mask", mask)
    cv2.imshow("morph", morph)
    cv2.imshow("fingers", fingers)
    
    key = cv2.waitKey(PERIOD) & 0xFF
    if key == ord('q'):
        break
    if key == ord('s'):
        with open("trackbar_memory.json", "w") as outfile:
            json.dump(trackbar_dict, outfile)
    if key == ord('p'):
        pause_flag = not pause_flag
        
cam.release()
cv2.destroyAllWindows()
cv2.waitKey(30)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is n

-1

## Изменения цвета движущегося объекта на неподвижном фоне

In [10]:
import cv2
import numpy as np

cam = cv2.VideoCapture('dancin.mp4')
fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)

hue_shift = 5

while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = fgbg.apply(frame)
    
    # Deleting background
    moving = cv2.bitwise_and(hsv, hsv, mask=mask)
    
    h, s, v = cv2.split(moving)
    h = (h.astype(np.int16) + hue_shift) % 180  # New hue
    h = h.astype(np.uint8)
    modified_hsv = cv2.merge([h, s, v])
    
    modified_frame = cv2.cvtColor(modified_hsv, cv2.COLOR_HSV2BGR)
    
    # Merge with original
    frame[mask > 0] = modified_frame[mask > 0]
    
    cv2.imshow('Result', frame)
    hue_shift = (hue_shift + 2) % 180
    
    key = cv2.waitKey(PERIOD) & 0xFF
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(30)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is not the object's thread (0x1d3c5910).
Cannot move to target thread (0x1e8b36e0)

QObject::moveToThread: Current thread (0x1e8b36e0) is n

-1